# Interactive Plotting 

###  Example from Anne:

In [ ]:
# Example downloading CMIP5 data.
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'projections-cmip5-monthly-single-levels',
    {
        'variable':[
            '2m_temperature','mean_precipitation_flux'
        ],
        'model':'noresm1_m',
        'experiment':'historical',
        'ensemble_member':'r1i1p1',
        'period':'185001-200512',
        'format':'tgz'
    },
    'cmip5_sfc_monthly_1850-200512.zip')

# Uncompress
import zipfile
zip_ref = zipfile.ZipFile('cmip5_sfc_monthly_1850-200512.zip', 'r')
# extract all the files in the directory /opt/uio/GyHoCd/data/
zip_ref.extractall('/opt/uio/GyHoCd/data/cmip5/')
zip_ref.close()

In [ ]:
path = ""

In [ ]:
import xarray as xr

# use open_mfdataset to open several netCDF files 
dset = xr.open_mfdataset("/opt/uio/GyHoCd/data/cmip5/*.nc")
print(dset)

In [1]:
path= '/home/hanna/Desktop/master_thesis/era/'

In [2]:
import glob

In [4]:
files = glob.glob(path + "*.nc")

In [5]:
from sclouds import helpers

In [6]:
helpers.DATA_REPO

'/uio/lagringshotell/geofag/students/metos/hannasv/data_processed/'

In [7]:
mapping = {"r":'relative_humidity_Europe.nc',
             "q":'specific_humidity_Europe.nc',
             "tcc":'total_cloud_cover_Europa.nc',
             "sp":'surface_pressure_Europa.nc',
             "t2m":'temperature_2m_Europa.nc'}

In [8]:
def get_season(data, season):
    """ OBS: only usefull after merge ts.
    Returns the xarray dataset containing only one season.
    """
    data = data.groupby('time.season')
    for group in data:
        key, dataset = group
        if key == season:
            return dataset

In [17]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
# timeslider 
timeslider = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Timeslider:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


# widget for variables
select_variable = widgets.Dropdown(
    options=['r', 'q', 'tcc', 'sp', 't2m'],
    value='t2m', # Think this is init
    description='Variable:',
    disabled=False,
)

# Widget for season
select_season = widgets.Dropdown(
    options=['DJF', 'MAM', 'JJA', 'SON', 'all'],
    value='all', # Think this is init
    description='Season:',
    disabled=False,
)


def generate_plot(var, ts, season):
    """  """
    # the change is the new variable
    filename = path + mapping[var]
    fig = plt.figure(figsize=[15,10])
    ax = fig.add_subplot(111, projection = ccrs.Orthographic(central_longitude=20, central_latitude=40))

    # TODO the 
    dset = xr.open_dataset(filename)
    if season != "all":
        dset = get_season(dset, season)
        
    #print(dset)
    if var == "r" or var == "q":
        # Humidities are multilevel parameters.
        dset[var][ts][0][:][:].plot(ax=ax,cmap='jet', transform=ccrs.PlateCarree())
    else:
        # Surface variables is single level parameters.
        dset[var][ts][:][:].plot(ax=ax,cmap='jet', transform=ccrs.PlateCarree())
    ax.coastlines()
    return fig

interact_plot = widgets.interact(generate_plot, 
                                 ts = timeslider, 
                                 season = select_season, 
                                 var = select_variable);

interactive(children=(Dropdown(description='Variable:', index=4, options=('r', 'q', 'tcc', 'sp', 't2m'), value…